In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', 50)
pd.get_option("display.max_columns")

50

In [2]:
train=pd.read_csv("train.csv",sep=",")
train.head()
test=pd.read_csv("test.csv",sep=",")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
train_X=train.loc[:,['Survived','Pclass','Age','Fare','SibSp','Parch','Sex','Embarked']]
train_X=train_X.replace('female',1).replace('male',0)
train_X.head()

,Survived,Pclass,Age,Fare,SibSp,Parch,Sex,Embarked
0,0,3,22.0,7.2500,1,0,0,S
1,1,1,38.0,71.2833,1,0,1,C
2,1,3,26.0,7.9250,0,0,1,S
3,1,1,35.0,53.1000,1,0,1,S
4,0,3,35.0,8.0500,0,0,0,S


In [4]:

test_X=test.loc[:,['Pclass','Age','Fare','SibSp','Parch','Sex','Embarked']]


In [5]:
train_X=train_X.replace('S',3).replace('C',2).replace('Q',1)
train_X.head()
#各列に欠損値はいくつあるか
train_X.isnull().any()
train_X.isna().sum()


Survived      0
Pclass        0
Age         177
Fare          0
SibSp         0
Parch         0
Sex           0
Embarked      2
dtype: int64

In [6]:
#train_XのAgeとEmbarke dにNanがあるので、Ageは平均で埋めてEmbarkedは欠損値のあるものは排除する。

#skleanでもNanを埋めることができるがこれをすると欠損値を平均で埋めるけど結果がnp.arrayになってしまう
#from sklearn.impute import SimpleImputer
#imp=SimpleImputer(strategy='mean')
#train_X=imp.fit_transform(train_X)

#そのため一つ一つカラムを処理しておく
train_X=train_X.fillna({'Age': '30'})
train_X=train_X[train_X['Embarked'].isna() == False]


In [7]:
test_X.isna().sum()

Pclass       0
Age         86
Fare         1
SibSp        0
Parch        0
Sex          0
Embarked     0
dtype: int64

In [8]:
test_X.describe()

,Pclass,Age,Fare,SibSp,Parch
count,418.000000,332.000000,417.000000,418.000000,418.000000
mean,2.265550,30.272590,35.627188,0.447368,0.392344
std,0.841838,14.181209,55.907576,0.896760,0.981429
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,7.895800,0.000000,0.000000
50%,3.000000,27.000000,14.454200,0.000000,0.000000
75%,3.000000,39.000000,31.500000,1.000000,0.000000
max,3.000000,76.000000,512.329200,8.000000,9.000000


In [9]:
#test_Xについても同様。一つ一つカラムを処理しておく
test_X=test_X.replace('female',1).replace('male',0)
test_X=test_X.replace('S',3).replace('C',2).replace('Q',1)
test_X=test_X.fillna({'Age': '27','Fare':'14.5'})


In [10]:
from sklearn import tree

In [11]:
#決定木、線形回帰ともにnp.ndarrayを引数にとるからnd.arrayに変換する
target = train_X["Survived"].values
features_one = train_X[["Pclass", "Sex", "Age", "Fare"]].values
 
# 決定木の作成
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)
 
# 「test」の説明変数の値を取得
test_features = test_X[["Pclass", "Sex", "Age", "Fare"]].values
 
# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = my_tree_one.predict(test_features)

In [12]:
PassengerId = np.array(test["PassengerId"]).astype(int)

#my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])

#my_tree_one.csvとして書き出し
my_solution.to_csv("my_ans.csv", index_label = ["PassengerId"])



In [13]:
#線形回帰
from sklearn.linear_model import LinearRegression
model=LinearRegression(fit_intercept=True)

In [14]:
model.fit(features_one,target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
predictions = model.predict(test_features)

In [16]:
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = 0

In [17]:
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution2 = pd.DataFrame(predictions, PassengerId, columns = ["Survived"])

In [18]:
my_solution2=my_solution2.astype(int)
my_solution2


,Survived
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0
1308,0


In [19]:
# my_tree_one.csvとして書き出し
my_solution2.to_csv("my_ans2.csv", index_label = ["PassengerId"])

In [20]:
#そのほかのやり方を考える。


#Embarkedと性別についてワンホットエンコーディング
train_X=pd.get_dummies(train, columns=['Embarked'])
train_X=pd.get_dummies(train_X, columns=['Sex'])
#年齢と運賃のNULLを中央値で埋める
train_X=train_X.fillna({'Age': '27','Fare':'14.5'})


In [21]:
train_X.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22,1,0,A/5 21171,7.2500,NaN,0,0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38,1,0,PC 17599,71.2833,C85,1,0,0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",26,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35,1,0,113803,53.1000,C123,0,0,1,1,0
4,5,0,3,"Allen, Mr. William Henry",35,0,0,373450,8.0500,NaN,0,0,1,0,1


In [22]:
#結果データと特徴データに分けるため抜き出し
ans_X=train_X['Survived']
features_X = train_X[["Pclass", "Sex_female","Sex_male", "Age", "Fare","Embarked_C","Embarked_Q","Embarked_S"]].values

In [23]:
#Embarkedと性別についてワンホットエンコーディング
test_X=pd.get_dummies(test, columns=['Embarked'])
test_X=pd.get_dummies(test_X, columns=['Sex'])
#年齢と運賃のNULLを中央値で埋める
test_X=test_X.fillna({'Age': '27','Fare':'14.5'})
#結果データと特徴データに分けるため抜き出し
test_features_X = test_X[["Pclass", "Sex_female","Sex_male", "Age", "Fare","Embarked_C","Embarked_Q","Embarked_S"]].values

In [24]:
#線形回帰
model.fit(features_X,ans_X)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:
predictions = model.predict(test_features_X)

In [26]:
test_features_X.shape

(418, 8)

In [27]:
PassengerId = np.array(test["PassengerId"]).astype(int)
PassengerId.shape

(418,)

In [28]:
predictions = model.predict(test_features_X)
predictions[predictions >= 0.4] = 1
predictions[predictions < 0.4] = 0
predictions=predictions.astype(int)

In [29]:
#ans_3=pd.Dataframe
ans5=pd.DataFrame(predictions,PassengerId,columns=['Survived'])
ans5.shape

(418, 1)

In [30]:
# my_tree_one.csvとして書き出し
ans5.to_csv("my_ans5.csv", index_label = ["PassengerId"])

In [31]:
from sklearn.model_selection import train_test_split

#train_test_split でtrainとtestデータに分割する
Xtrain,Xtest,ytrain,ytest=train_test_split(features_X,ans_X,random_state=1)

In [32]:
model.fit(Xtrain,ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
predictions = model.predict(Xtest)
predictions[predictions >= 0.4] = 1
predictions[predictions < 0.4] = 0
predictions=predictions.astype(int)


In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,predictions)
#accuracyが高くていけそうなのでこのまま予測して行こうかな

0.7982062780269058

In [42]:
test_features=test_features.loc[:,]

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,892,3,"Kelly, Mr. James",34.5,0,0,330911,7.8292,NaN,0,1,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",47,1,0,363272,7,NaN,0,0,1,1,0
2,894,2,"Myles, Mr. Thomas Francis",62,0,0,240276,9.6875,NaN,0,1,0,0,1
3,895,3,"Wirz, Mr. Albert",27,0,0,315154,8.6625,NaN,0,0,1,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22,1,1,3101298,12.2875,NaN,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",27,0,0,A.5. 3236,8.05,NaN,0,0,1,0,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",39,0,0,PC 17758,108.9,C105,1,0,0,1,0
415,1307,3,"Saether, Mr. Simon Sivertsen",38.5,0,0,SOTON/O.Q. 3101262,7.25,NaN,0,0,1,0,1
416,1308,3,"Ware, Mr. Frederick",27,0,0,359309,8.05,NaN,0,0,1,0,1


In [43]:
#決定木はnp.ndarrayを引数にとるからnd.arrayに変換する
target = ans_X.values
features_one = features_X
 
# 決定木の作成
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)
 
# 「test」の説明変数の値を取得
test_features  = test_X[["Pclass", "Sex_female","Sex_male", "Age", "Fare","Embarked_C","Embarked_Q","Embarked_S"]].values
 
# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction2 = my_tree_one.predict(test_features)

In [44]:
my_prediction2

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [46]:
# my_tree_one.csvとして書き出し
ans6=pd.DataFrame(my_prediction2,PassengerId,columns=['Survived'])
ans6.shape
ans6.to_csv("my_ans6.csv", index_label = ["PassengerId"])